In [1]:
import configparser
import psycopg2

In [1]:
########

123

In [9]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [3]:
config.sections()

['CLUSTER', 'IAM_ROLE', 'S3']

In [4]:
for item in config['CLUSTER']:
    print(item)

host
db_name
db_user
db_password
db_port


In [2]:
#("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))

In [10]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [43]:
# execute to rollback a transaction, i.e. when postgresql throws an error
cur.execute("rollback");

In [7]:
## run a simple query, to convince myself I'm actually connected to the redshift cluster
query = """
SELECT DISTINCT tablename
FROM pg_table_def limit 5
"""
cur.execute(query)


cur.fetchall()

[('padb_config_harvest',),
 ('pg_aggregate',),
 ('pg_am',),
 ('pg_amop',),
 ('pg_amproc',)]

In [8]:
## run a simple query, to convince myself I'm actually connected to the redshift cluster
query = """ 
select *
from pg_aggregate
limit 5;
"""
cur.execute(query)


cur.fetchall()

[('pg_catalog.avg', 'int8_avg_accum', 'int8_avg', 1016, '{0,0}'),
 ('pg_catalog.avg', 'int4_avg_accum', 'int8_avg', 1016, '{0,0}'),
 ('pg_catalog.avg', 'int2_avg_accum', 'int8_avg', 1016, '{0,0}'),
 ('pg_catalog.avg', 'numeric_accum', 'numeric_avg', 1231, '{0,0,0}'),
 ('pg_catalog.avg', 'float4_accum', 'float8_avg', 1022, '{0,0,0}')]

In [28]:
conn.commit()

In [3]:
## Or connect using the sql extention
DWH_ENDPOINT=
DWH_DB=
DWH_USER=
DWH_DB_PASSWORD=
DWH_PORT=
%reload_ext sql
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
%sql $conn_string

'Connected: dwhuser@dwh'

In [8]:
staging_events_table_create= ("""
    CREATE TABLE IF NOT EXISTS staging_events (
        event_id    BIGINT IDENTITY(0,1),
        artist      VARCHAR,
        auth        VARCHAR,
        firstName   VARCHAR,
        gender      VARCHAR,
        itemInSession VARCHAR,
        lastName    VARCHAR,
        length      VARCHAR,
        level       VARCHAR,
        location    VARCHAR,
        method      VARCHAR,
        page        VARCHAR,
        registration VARCHAR,
        sessionId   INTEGER SORTKEY DISTKEY,
        song        VARCHAR,
        status      INTEGER,
        ts          BIGINT,
        userAgent   VARCHAR,
        userId      INTEGER
    );
""")

In [10]:
staging_events_table_create

'\n    CREATE TABLE IF NOT EXISTS staging_events (\n        event_id    BIGINT IDENTITY(0,1),\n        artist      VARCHAR,\n        auth        VARCHAR,\n        firstName   VARCHAR,\n        gender      VARCHAR,\n        itemInSession VARCHAR,\n        lastName    VARCHAR,\n        length      VARCHAR,\n        level       VARCHAR,\n        location    VARCHAR,\n        method      VARCHAR,\n        page        VARCHAR,\n        registration VARCHAR,\n        sessionId   INTEGER SORTKEY DISTKEY,\n        song        VARCHAR,\n        status      INTEGER,\n        ts          BIGINT,\n        userAgent   VARCHAR,\n        userId      INTEGER\n    );\n'

In [11]:
cur.execute(staging_events_table_create)
conn.commit()

In [34]:
cur.execute("rollback");

## check if my custom table actually exists
query = """ 
select * from PG_TABLE_DEF where tablename = 'staging_events' limit 5;
"""
cur.execute(query)


cur.fetchall()

[('public', 'staging_events', 'event_id', 'bigint', 'az64', False, 0, False),
 ('public',
  'staging_events',
  'artist',
  'character varying(256)',
  'lzo',
  False,
  0,
  False),
 ('public',
  'staging_events',
  'auth',
  'character varying(256)',
  'lzo',
  False,
  0,
  False),
 ('public',
  'staging_events',
  'firstname',
  'character varying(256)',
  'lzo',
  False,
  0,
  False),
 ('public',
  'staging_events',
  'gender',
  'character varying(256)',
  'lzo',
  False,
  0,
  False)]

In [18]:
staging_songs_table_create = ("""
    CREATE TABLE IF NOT EXISTS staging_songs (
        artist_id VARCHAR,
        artist_latitude FLOAT,
        artist_location TEXT,
        artist_longitude FLOAT,
        artist_name VARCHAR,
        duration FLOAT,
        num_songs INT,
        song_id VARCHAR,
        title VARCHAR,
        year INT
    )
""")

In [44]:
cur.execute(staging_songs_table_create)


In [3]:
## deleting a table
qry = "drop table staging_logs;"
%sql $qry


UsageError: Line magic function `%sql` not found.


In [67]:
#{"artist":null,"auth":"Logged In","firstName":"Walter","gender":"M","itemInSession":0,"lastName":"Frye","length":null,"level":"free","location":"San Francisco-Oakland-Hayward, CA","method":"GET","page":"Home","registration":1540919166796.0,"sessionId":38,"song":null,"status":200,"ts":1541105830796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.143 Safari\/537.36\"","userId":"39"}

staging_logs_table_create = ("""
    CREATE TABLE IF NOT EXISTS staging_logs (
        artist VARCHAR,
        auth VARCHAR,
        firstName VARCHAR,
        gender VARCHAR,
        itemInSession INT,
        lastName VARCHAR,
        length FLOAT,
        level VARCHAR,
        location VARCHAR,
        method VARCHAR,
        page VARCHAR,
        registration FLOAT,
        sessionId INT,
        song VARCHAR,
        status INT,
        ts BIGINT,
        userAgent VARCHAR,
        userId VARCHAR
    )
""")

print(staging_logs_table_create)


    CREATE TABLE IF NOT EXISTS staging_logs (
        artist VARCHAR,
        auth VARCHAR,
        firstName VARCHAR,
        gender VARCHAR,
        itemInSession INT,
        lastName VARCHAR,
        length FLOAT,
        level VARCHAR,
        location VARCHAR,
        method VARCHAR,
        page VARCHAR,
        registration FLOAT,
        sessionId INT,
        song VARCHAR,
        status INT,
        ts BIGINT,
        userAgent VARCHAR,
        userId VARCHAR
    )



In [68]:
cur.execute("rollback");
cur.execute(staging_logs_table_create)

In [4]:
## check if my custom table actually exists
qry = "select * from staging_songs limit 3;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [20]:
## check if my custom table actually exists
qry = "select * from PG_TABLE_DEF where tablename = 'staging_logs' ;"
%sql $qry


UsageError: Line magic function `%sql` not found.


In [80]:
LOG_DATA = config.get("S3", "LOG_DATA")
LOG_DATA


"'s3://udacity-dend/log_data'"

In [107]:
ARN=
SONG_DATA='s3://udacity-dend/song_data'
SONG_DATA='s3://udacity-dend/song_data/A/A'
LOG_DATA='s3://udacity-dend/log_data'

In [5]:
staging_songs_copy = ("""COPY staging_songs FROM '{}'
credentials '{}'
region 'us-west-2'
json 'auto'
""").format(SONG_DATA, ARN)
print(staging_songs_copy)

NameError: name 'SONG_DATA' is not defined

In [6]:
staging_logs_copy = ("""COPY staging_logs FROM '{}'
credentials 'aws_iam_role={}'
region 'us-west-2'
json 'auto'
""").format(LOG_DATA, ARN)
print(staging_songs_copy)

NameError: name 'LOG_DATA' is not defined

In [71]:
cur.execute(staging_logs_copy)

In [7]:
qry = "select * from staging_songs limit 5;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [8]:
## results are easier to see using the ipython sql extension
qry = "select * from staging_logs limit 5";
%sql $qry

UsageError: Line magic function `%sql` not found.


In [9]:
qry = "select * from stl_load_errors limit 100;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [91]:
from sql_queries import copy_table_queries

In [10]:
for i in copy_table_queries:
    print(i)

NameError: name 'copy_table_queries' is not defined

In [ ]:
########################################################################
# STAR SCHEMA

In [14]:

user_table_create = ("""
    CREATE TABLE IF NOT EXISTS users (
        user_id VARCHAR, 
        first_name VARCHAR, 
        last_name VARCHAR, 
        gender VARCHAR, 
        level VARCHAR
    )
""")

In [15]:
cur.execute("rollback");
cur.execute(user_table_create)

In [11]:
## check if my custom table actually exists
qry = "select * from PG_TABLE_DEF where tablename = 'users' ;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [12]:
qry = "insert into users select distinct userid,firstname,lastname,gender,level from staging_events;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [13]:
qry = "select * from users limit 2;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [14]:
qry = "select * from staging_events limit 2;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [15]:
artists_table_create = ("""
    CREATE TABLE IF NOT EXISTS artists (
        artist_id VARCHAR, 
        name VARCHAR, 
        location VARCHAR, 
        lattitude VARCHAR, 
        longitude VARCHAR
    )
""")
cur.execute("rollback");
cur.execute(artists_table_create)
qry = "select * from artists limit 1;"
%sql $qry

NameError: name 'cur' is not defined

In [16]:
qry = "insert into artists select distinct artist_id, artist_name, artist_location, artist_latitude, artist_longitude from staging_songs;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [17]:
qry = "select * from artists limit 5;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [ ]:
songs - songs in music database
song_id, title, artist_id, year, duration

artists - artists in music database
artist_id, name, location, lattitude, longitude

time - timestamps of records in songplays broken down into specific units
start_time, hour, day, week, month, year, weekday

In [18]:
qry = "select * from staging_songs limit 1;"
%sql $qry

UsageError: Line magic function `%sql` not found.


In [28]:
song_table_create = ("""
    CREATE TABLE IF NOT EXISTS songs (
        song_id VARCHAR, 
        title VARCHAR, 
        artist_id VARCHAR, 
        year VARCHAR
    )
""")

In [29]:
cur.execute("rollback");
cur.execute(song_table_create)

In [19]:
#song_id, title, artist_id, year, duration
qry= "insert into songs select song_id, title, year, duration from staging_songs"
%sql $qry

UsageError: Line magic function `%sql` not found.
